## The Efficientnet model development and fine tuning

in this notebook we will load and add our layers to the base model of the pre-trained EfficientnetB7 model, and fine tune the following parameters of the model:
1. **Optimizers** - We will use only 5 optimizers to see which one works best, namely; 1. `SGD`,2. `RMSprop`,3. `Adam`,4. `Adagrad`,5. `Adadelta`. the best optimizer will be used in the New model
2. **Number of epochs** - We will also use only 4 epochs to choose the best performer namely; `1, 2, 5, 10`.
3. **Batch size** - we will also use these batch sizes to choose the optimum batch size, namely; `8, 16`. 
4. **Dropout** - we will optimise the dropout of the model, values `0.5, 0.6, 0.7, 0.8, 0.9`

> **Section one:** we will create common function and also create our base model, with the following initial parameters: **Batch size** `8`, **Optimizer** `Adam`, **Number of epochs** `4`, **dropout value** `0.5`. 

# **Import libraries that we will use**

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import efficientnet.keras as efn 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook
import time
import tqdm.gui as tqdm
import tqdm.notebook as tqdm

In [3]:
from tensorflow.keras.layers import Activation, Dense

# **Common functions**

In [4]:
# common functions that i will use

def get_data_generator(train_data_path, \
                       val_data_path, \
                       targetsize, \
                       classmode, \
                       batchsize):
    """
    This function is a data generator function for train, validation, and testing data
    Inputs
        train_data_path   : train data path for the dataset 
        val_data_path     : validation data path for the dataset
        targetsize        : target size for the generator to resize all images to, (224,224)
        classmode         : class mode, 'categorical'
        batchsize         : batch size
    Outputs
        train_generator   : generated train data
        val_generator     : generated validation data
    
    """
    train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    
    train_generator=train_datagen.flow_from_directory(train_data_path, # this is where you specify the path to the main data folder
                                                 target_size=targetsize,
                                                 color_mode='rgb',
                                                 batch_size=batchsize,
                                                 class_mode=classmode,
                                                 shuffle=True)
    
    val_generator=val_datagen.flow_from_directory(val_data_path, # this is where you specify the path to the main data folder
                                                 target_size=targetsize,
                                                 color_mode='rgb',
                                                 batch_size=batchsize,
                                                 class_mode=classmode,)
 
    return train_generator,val_generator   
    


def train_evaluate_the_model(train_generator, \
                             val_generator, \
                             optimizer, \
                             epochs, \
                             dropout_value, \
                             TheModel):
    '''
    train the model, do predictions, and do evaluation and return the accuracy of the model
    Inputs
        train_generator : generated train data
        val_generator   : generated validation data
        optimizer       : the optimizer method used to compile the model
        epochs          : the epochs of the model use to fit the model
        dropout_value   : the dropout value
        TheModel        : The pretrained model loaded,
    Outputs
        accuracy        : The accuracy of the model
    '''
    # load pretrained model and add layers on top of the model
    x = TheModel.output
    x = tf.keras.layers.Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout_value)(x)
    predictions = Dense(4, activation="sigmoid")(x)
    model = Model(inputs = TheModel.input, outputs = predictions)
    
    # freeze base layers for training
    for layer in TheModel.layers:
        layer.trainable = False
    
    # compile the model
    model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    step_size_train=train_generator.n//train_generator.batch_size
    # fit the model
    r = model.fit_generator(generator=train_generator,
                        validation_data=val_generator,
                        steps_per_epoch=step_size_train,
                        epochs=epochs)
    # valuate the model
    scores = model.evaluate(val_generator)
    accuracy = scores[1]*100
    
    return accuracy, model

# **Load and generate data**

In [5]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

'''
Initial and constant parameters for the model
initial parameters
       batch_size    : the initial batch size we will create our base model with
Constant parameters
       target_size   : Constant image target size for the whole notebook which is (224,224)
       class_mode    : class mode we will use for the whole notebook, which is 'categorical'
'''
# constant parameters
targetsize_constant = (224,224)
classmode_constant = 'categorical'

# initial parameters
batchsize_initial = 8
epochs_initial = 4
dropout_value_initial = 0.5
optimizer_initial = 'Adam'

# Data paths
train_dataset_path = 'Dataset/Training/'
validation_dataset_path = 'Dataset/Validation/'
test_dataset_path = 'Dataset/Testing/'


train_generator_initial = train_datagen.flow_from_directory(train_dataset_path, # this is where you specify the path to the main data folder
                                                 target_size=targetsize_constant,
                                                 color_mode='rgb',
                                                 batch_size=batchsize_initial,
                                                 class_mode=classmode_constant,
                                                 shuffle=True)
val_generator_initial = val_datagen.flow_from_directory(validation_dataset_path, # this is where you specify the path to the main data folder
                                                 target_size=targetsize_constant,
                                                 color_mode='rgb',
                                                 batch_size=batchsize_initial,
                                                 class_mode=classmode_constant,
                                                 shuffle=True)


Found 2870 images belonging to 4 classes.
Found 2870 images belonging to 4 classes.


In [6]:
# display classes found in the data
train_generator_initial.class_indices

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}

# **Import the EfficientnetB7 base model**

In [7]:
from tensorflow.keras.applications import EfficientNetB7

In [8]:
image_size = [224,224] # choose image size
# import the base model
efnB7 = tf.keras.applications.efficientnet.EfficientNetB7(input_shape= image_size+[3],weights='imagenet',include_top=False)

# storing the base model in the kernel for later use to avoid loading many times
efnB7_basemodel = efnB7


#### Add layers on top of the base model

In [9]:
# initial parameter value
dropout_value_initial = 0.1

# add layers
x = efnB7.output
x = tf.keras.layers.Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(dropout_value_initial)(x)
predictions = Dense(4, activation="sigmoid")(x)
model = Model(inputs = efnB7.input, outputs = predictions)

In [10]:
# freeze base layers for training
for layer in efnB7.layers:
    layer.trainable = False

# **Compile the model**

In [11]:
# initial optimizer
optimizer_initial = 'Adam'

# compile the model
model.compile(optimizer=optimizer_initial,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# **Fit the model**

In [12]:
# initial parameters
epochs_initial = 4
step_size_train_initial=train_generator_initial.n//train_generator_initial.batch_size # computing the steps size per epoch

tic = time.time()
# fit the model
r = model.fit_generator(generator=train_generator_initial,
                        validation_data=val_generator_initial,
                        steps_per_epoch=step_size_train_initial,
                        epochs=epochs_initial)

toc = time.time()
print("Minutes taken = " + str((toc-tic)/60.0))

C:\Users\ganas\AppData\Local\Temp\ipykernel_8608\768208122.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(generator=train_generator_initial,


Epoch 1/4
358/358 [==============================] - 3304s 9s/step - loss: 3.0046 - accuracy: 0.5709 - val_loss: 0.8188 - val_accuracy: 0.6812
Epoch 2/4
358/358 [==============================] - 58746s 165s/step - loss: 0.8639 - accuracy: 0.6782 - val_loss: 0.4725 - val_accuracy: 0.8275
Epoch 3/4
358/358 [==============================] - 3303s 9s/step - loss: 0.6665 - accuracy: 0.7502 - val_loss: 0.4789 - val_accuracy: 0.8010
Epoch 4/4
358/358 [==============================] - 3293s 9s/step - loss: 0.6631 - accuracy: 0.7572 - val_loss: 0.4439 - val_accuracy: 0.8303
Minutes taken = 1144.150994348526


# **Evaluate the model**

In [13]:
# evaluation on validation data
scores = model.evaluate(val_generator_initial)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))


359/359 [==============================] - 1312s 4s/step - loss: 0.4439 - accuracy: 0.8303
evaluate accuracy: 83.03%


# **Section two:** We will start fine tunning

## **Tunning optimizers of the model**

In [14]:
param_label = 'optimizer'
param_list = ['Adam', 'SGD', 'RMSprop', 'Adagrad', 'Adadelta'] # ['Adam', 'SGD', 'RMSprop', 'Adagrad', 'Adadelta']

accuracy_table = {param_label: [], 'accuracy': []}
tic = time.time()
for param in tqdm.tqdm_notebook(param_list):
    # Train, and evaluate model
    accuracy, _ = train_evaluate_the_model(train_generator_initial,val_generator_initial,param,epochs_initial,dropout_value_initial,efnB7_basemodel) # replace epochs with 'epochs_initial'
    
    # Collect results
    accuracy_table[param_label].append(param)
    accuracy_table['accuracy'].append(accuracy)
    
accuracy_table = pd.DataFrame(accuracy_table) # convert the table to a dataframe
toc = time.time()
print("Minutes taken = " + str((toc-tic)/60.0))
accuracy_table # display the resullts

  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\ganas\AppData\Local\Temp\ipykernel_8608\1030359104.py:80: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(generator=train_generator,


Epoch 1/4
358/358 [==============================] - 4281s 12s/step - loss: 2.9061 - accuracy: 0.5531 - val_loss: 0.5983 - val_accuracy: 0.7801
Epoch 2/4
358/358 [==============================] - 5756s 16s/step - loss: 0.9169 - accuracy: 0.6674 - val_loss: 0.5646 - val_accuracy: 0.7878
Epoch 3/4
358/358 [==============================] - 3960s 11s/step - loss: 0.6793 - accuracy: 0.7449 - val_loss: 0.4450 - val_accuracy: 0.8275
Epoch 4/4
359/359 [==============================] - 1361s 4s/step - loss: 0.3594 - accuracy: 0.8679
Epoch 1/4
327/358 [==========================>...] - ETA: 1:40:42 - loss: 1.2483 - accuracy: 0.4946

In [ ]:
from pylab import rcParams

# plot results
rcParams['figure.figsize'] = 10,8 # width 10, height 8

ax = accuracy_table.plot(x='optimizer', y='accuracy',style='bx-', grid=True)
ax.set_xlabel("Optimizer")
ax.set_ylabel("accuracy)")

: 

In [ ]:
# Get optimum value for param
temp = accuracy_table[accuracy_table['accuracy'] == accuracy_table['accuracy'].max()]
optimizer_opt = temp[param_label].values[0]
print("max Accuracy = %0.3f" % accuracy_table['accuracy'].max())
print("optimum " + param_label + " = " + str(optimizer_opt))

# **Tunning dropout of the model**

In [ ]:
param_label = 'dropout_prob'
param_list = [0.5, 0.6, 0.7, 0.8, 0.9]


accuracy_table = {param_label: [], 'accuracy': []}
tic = time.time()
for param in tqdm.tqdm_notebook(param_list):
    # Train, predict and evaluate model
    accuracy, _ = train_evaluate_the_model(train_generator_initial,val_generator_initial,optimizer_initial,epochs_initial,param,efnB7_basemodel) # replace epochs with 'epochs_initial'
    
    # Collect results
    accuracy_table[param_label].append(param)
    accuracy_table['accuracy'].append(accuracy)
    
accuracy_table = pd.DataFrame(accuracy_table) # convert the table to a dataframe
toc = time.time()
print("Minutes taken = " + str((toc-tic)/60.0))
accuracy_table # display the resullts

In [ ]:
from pylab import rcParams

# plot results
rcParams['figure.figsize'] = 10,8 # width 10, height 8

ax = accuracy_table.plot(x='dropout_prob', y='accuracy',style='bx-', grid=True)
ax.set_xlabel("dropout_prob")
ax.set_ylabel("accuracy)")

In [ ]:
# Get optimum value for param
temp = accuracy_table[accuracy_table['accuracy'] == accuracy_table['accuracy'].max()]
dropout_opt = temp[param_label].values[0]
print("max Accuracy = %0.3f" % accuracy_table['accuracy'].max())
print("optimum " + param_label + " = " + str(dropout_opt))

# **Tuning batch size and epochs of the model**

In [ ]:
param_label = 'epochs'
param_list = [1,2,5,10] # [5, 10, 15, 20]

param2_label = 'batch_size'
param2_list = [8, 16] # [8, 16, 32] am using this batch sizes for now until we fix the accuracy issue, proper batch sizes to be use [8, 16, 32, 64, 128]

accuracy_table = {param_label: [], param2_label: [], 'accuracy': []}
tic = time.time()
for param in tqdm.tqdm_notebook(param_list):
    for param2 in tqdm_notebook(param2_list):
        
        # generate train and validation data
        train_generator,val_generator, _ = get_data_generator(train_dataset_path,validation_dataset_path,test_dataset_path, targetsize_constant, classmode_constant, param2)
        # train, predict and evaluate model
        accuracy, _ = train_evaluate_the_model(train_generator, val_generator, optimizer_initial, param,dropout_value_initial, efnB7_basemodel)
        
        # collect results
        accuracy_table[param_label].append(param)
        accuracy_table[param2_label].append(param2)
        accuracy_table['accuracy'].append(accuracy)
        
accuracy_table = pd.DataFrame(accuracy_table) # convert the table to a dataframe
toc = time.time()
print("Minutes taken = " + str((toc-tic)/60.0))
accuracy_table # display the resullts

In [ ]:
from pylab import rcParams

# Plot performance versus params
rcParams['figure.figsize'] = 10, 8 # width 10, height 8
temp = accuracy_table[accuracy_table[param2_label]==param2_list[0]]
ax = temp.plot(x=param_label, y='accuracy', style='bs-', grid=True)
legend_list = [param2_label + '_' + str(param2_list[0])]

color_list = ['r', 'g', 'k', '0.75']
for i in range(1,len(param2_list)):
    temp = accuracy_table[accuracy_table[param2_label]==param2_list[i]]
    ax = temp.plot(x=param_label, y='accuracy', color=color_list[i%len(color_list)], marker='s', grid=True, ax=ax)
    legend_list.append(param2_label + '_' + str(param2_list[i]))

ax.set_xlabel(param_label)
ax.set_ylabel("accuracy")
plt.rcParams.update({'font.size': 14})
plt.legend(legend_list, loc='center left', bbox_to_anchor=(1.0, 0.5)) # positions legend outside figure
# ax.set_xlim([10, 50])
# ax.set_ylim([0, 5])

In [ ]:
# Get optimum value for param and param2
temp = accuracy_table[accuracy_table['accuracy'] == accuracy_table['accuracy'].max()]
epochs_opt = temp[param_label].values[0]
batch_size_opt = temp[param2_label].values[0]
print("max Accuracy = %0.3f" % accuracy_table['accuracy'].max())
print("optimum " + param_label + " = " + str(epochs_opt))
print("optimum " + param2_label + " = " + str(batch_size_opt))

In [ ]:
######
dropout_opt = 0.6
batch_size_opt = 16
epochs_opt = 10
optimizer_opt = 'Adagrad'
#####

# **Collecting all optimum tunned parameters**

In [ ]:
d = {'param': ['optimizer', 'epochs', 'batch_size','dropout_prob'],
     'original': [optimizer_initial, epochs_initial, batchsize_initial,dropout_value_initial],
     'after_tuning': [optimizer_opt, epochs_opt, batch_size_opt,dropout_opt]}
tuned_params = pd.DataFrame(d)
tuned_params

# **Creating the final model with optimum parameters**

In [ ]:
# get train and validation data generator
train_generator_opt,val_generator_opt,test_generator_opt = get_data_generator(train_dataset_path,validation_dataset_path,test_dataset_path, targetsize_constant, classmode_constant, batch_size_opt)

# train and evaluate the model
accuracy, New_model = train_evaluate_the_model(train_generator_opt, val_generator_opt, optimizer_opt, epochs_opt,dropout_opt , efnB7_basemodel)


In [ ]:
accuracy

# **Save the model**

In [ ]:
New_model.save('brain_tumor_classification_efnB7_test10091335.hdf5') #28060904 - dd/mm/hours/minutes

# ---------------------------------------- END OF NOTEBOOK -----------------------------------
